In [ ]:
import os
import shutil
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random

# --- 설정값 ---
# 📝 원본 데이터셋의 최상위 폴더 경로 (이미 파일명이 정리되어 있는 폴더)
source_base_dir = 'D:/AiSchool/1stProject/New Data/네이밍/Train_naming'

# 📝 증강 결과를 저장할 최상위 폴더 경로
new_base_dir = 'D:/AiSchool/1stProject/New Data/네이밍/Train_augmentation'

# --- 함수 정의 (전역 스코프에 정의) ---

# 6. 무작위 노이즈 추가 함수 (PIL/Numpy 기반)
def add_noise(img):
    img_np = np.array(img, dtype=np.int16) 
    mean = 0
    sigma = 15
    gauss = np.random.normal(mean, sigma, img_np.shape)
    noisy_img_np = np.clip(img_np + gauss, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_img_np)

# 9. 전단 변형 (Shear)
def apply_shear(img):
    width, height = img.size
    shear_matrix = (1, 0.176, 0, 0, 1, 0)
    return img.transform((width, height), Image.Transform.AFFINE, shear_matrix, resample=Image.Resampling.BICUBIC)

# 헬퍼 함수: index를 인수로 받고, 증가시킨 index를 반환하도록 수정 (증강 파일 저장)
def augment_and_save(img_path, method_name, augment_func, extension, base_drug_name, new_folder_path, current_index):
    try:
        img = Image.open(img_path).convert('RGB')
        augmented_img = augment_func(img)
        
        new_number = str(current_index).zfill(3)
        
        # 증강 파일명 규칙: (약이름)_(넘버링)_(증강기법).확장자명
        new_name = f"{base_drug_name}_{new_number}_{method_name.lower()}{extension}"
        dst_file = os.path.join(new_folder_path, new_name)

        augmented_img.save(dst_file)
        # 성공적으로 저장되면 인덱스 1 증가
        return current_index + 1
    except Exception as e:
        print(f"❌ 증강 실패 ({img_path}, {method_name}): {e}")
        return current_index # 실패 시 인덱스 증가시키지 않음

# -------------------------------------------------------------
# 🌟 메인 처리 루프 🌟
# -------------------------------------------------------------
print(f"✅ 데이터 증강 및 원본 복사 처리 시작: {source_base_dir}\n")

pill_folders = [d for d in os.listdir(source_base_dir) if os.path.isdir(os.path.join(source_base_dir, d))]

if not pill_folders:
    print("❌ 오류: 원본 최상위 폴더 내에 처리할 하위 폴더가 없습니다.")
    exit()

total_files_processed = 0

for long_folder_name in pill_folders:
    print(f"\n--- 🧪 폴더 처리 시작: {long_folder_name} ---")
    
    # 1. 경로 설정
    source_folder_path = os.path.join(source_base_dir, long_folder_name)
    new_folder_path = os.path.join(new_base_dir, long_folder_name)
    
    # 2. 🌟 순수 약 이름 추출 (파일명 접두사로 사용)
    parts = long_folder_name.split('_') 
    
    if len(parts) >= 2:
        base_drug_name = parts[1].strip()
    else:
        base_drug_name = long_folder_name.strip()
        
    if not base_drug_name:
        base_drug_name = "NewFiles"
        
    # 대상 폴더 생성
    os.makedirs(new_folder_path, exist_ok=True)
    print(f"   ✅ 파일명 접두사 (추출된 약 이름): {base_drug_name}")
    print(f"   ✅ 저장될 폴더: {new_folder_path}")

    # 파일 목록 가져오기 (원본 파일들, 이름이 (약이름)_(넘버링)_original.확장자 형태로 정리되어 있다고 가정)
    file_names = sorted([f for f in os.listdir(source_folder_path) if os.path.isfile(os.path.join(source_folder_path, f))])

    # -------------------------------------------------------------
    # 1. 원본 파일 복사 (이름 변경 없이 그대로 복사)
    # -------------------------------------------------------------
    files_copied_count = 0
    for old_name in file_names:
        src_file = os.path.join(source_folder_path, old_name)
        dst_file = os.path.join(new_folder_path, old_name) # 📌 이름 변경 없이 그대로 사용
        
        try:
            shutil.copy2(src_file, dst_file)
            files_copied_count += 1
        except Exception as e:
            print(f"❌ 원본 복사 실패 ({old_name}): {e}")

    # -------------------------------------------------------------
    # 2. 데이터 증강 및 저장 (10가지 기법)
    # -------------------------------------------------------------
    
    # 증강 파일은 원본 파일의 넘버링을 이어받아 시작합니다.
    # 즉, 원본 파일이 N개라면 증강 파일의 넘버링은 N+1부터 시작합니다.
    local_index = len(file_names) + 1 
    files_augmented_count = 0
    
    for old_name in file_names:
        src_file = os.path.join(source_folder_path, old_name)
        # 확장자는 기존 파일명에서 추출하지 않고, old_name에서 다시 추출
        ext_part = os.path.splitext(old_name)[1] 

        # --- 10가지 기법 적용 ---
        random_number = random.choice([15, 30, 45, 60, 90])
        
        # 🎯 함수를 호출하고 반환된 새 인덱스 값으로 local_index를 업데이트
        local_index = augment_and_save(src_file, 'flip_H', lambda img: img.transpose(Image.FLIP_LEFT_RIGHT), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'rot_R', lambda img: img.rotate(random_number, resample=Image.Resampling.BICUBIC), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'bright_15', lambda img: ImageEnhance.Brightness(img).enhance(1.5), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'sharp_20', lambda img: ImageEnhance.Sharpness(img).enhance(2.0), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'blur_G', lambda img: img.filter(ImageFilter.GaussianBlur(radius=1.5)), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'noise_G', add_noise, ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'flip_V', lambda img: img.transpose(Image.FLIP_TOP_BOTTOM), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'color_05', lambda img: ImageEnhance.Color(img).enhance(0.5), ext_part, base_drug_name, new_folder_path, local_index)
        local_index = augment_and_save(src_file, 'shear_10', apply_shear, ext_part, base_drug_name, new_folder_path, local_index)

        files_augmented_count += 10 

    total_files_processed += (files_copied_count + files_augmented_count)
    print(f"   ✅ 원본 파일 복사 수: {files_copied_count}개")
    print(f"   ✅ 증강된 파일 수: {files_augmented_count}개 (넘버링: {files_copied_count + 1}부터 {local_index - 1}까지)")
    print(f"   ✅ 최종 파일 수: {files_copied_count + files_augmented_count}개")


print("\n---")
print(f"🎉 모든 폴더 처리 완료. 총 {total_files_processed}개의 파일이 복사 및 증강되었습니다.")

✅ 데이터 증강 및 원본 복사 처리 시작: D:/AiSchool/1stProject/New Data/네이밍/Train_naming



FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'D:/AiSchool/1stProject/New Data/네이밍/Train_naming'